In [1]:
import os
from pydub import AudioSegment
import librosa
import soundfile as sf
from matplotlib import pyplot as plt
import librosa.display
import numpy as np
import io
from keras.models import load_model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint


C:\Users\elsha\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


# Frequency Masking

In [ ]:
import librosa
import numpy as np

# Define function for frequency masking
def freq_mask(spec, F=30, num_masks=1, replace_with_zero=True):
    """
    Apply frequency masking to a spectrogram.

    Args:
    spec (ndarray): Input spectrogram of shape (freq_bins, time_frames)
    F (int): Maximum number of consecutive frequency bins to mask
    num_masks (int): Number of masks to apply
    replace_with_zero (bool): Whether to replace masked values with zero or random values

    Returns:
    masked_spec (ndarray): Masked spectrogram of shape (freq_bins, time_frames)
    """
    masked_spec = spec.copy()
    num_bins, num_frames = masked_spec.shape

    for i in range(num_masks):
        f = np.random.randint(low=0, high=F)
        start = np.random.randint(low=0, high=num_bins - f)
        masked_spec[start:start + f, :] = 0.0

        if not replace_with_zero:
            mask_value = np.random.uniform(low=masked_spec.min(), high=masked_spec.max())
            masked_spec[start:start + f, :] = mask_value

    return masked_spec


folder_path = r"D:\Education\Semester 6\CSE 321 Project Based Learning on CSE\Composers Classification\Mel-Spectrogram\Train"
Out = r"D:\Education\Semester 6\CSE 321 Project Based Learning on CSE\Composers Classification\Experiment 5\Train"

for folder in os.listdir(folder_path):
    comp_in = os.path.join(folder_path, folder)
    comp_out = os.path.join(Out, folder)
    for file_name in os.listdir(comp_in):
        file_path = os.path.join(comp_in, file_name)
        mel = np.load(file_path)
        
        # Apply frequency masking
        masked_spec = freq_mask(mel)
        
        out_path = os.path.join(comp_out, file_name)
        np.save(out_path, masked_spec)
        

In [ ]:
def preparing_data(data_path):
    # define path to MFCC data and number of classes
    num_classes = 8
    counter = 1

    # load and preprocess data
    X = []
    y = []
    for composer_idx in range(num_classes):
        composer_path = os.path.join(data_path, f'{composer_idx}')
        for file in os.listdir(composer_path):
            file_path = os.path.join(composer_path, file)
            mel = np.load(file_path, allow_pickle=True)
            if (mel.shape == (128, 216)):
                X.append(mel)
                y.append(composer_idx)
                counter += 1
            else:
                print(":(")

    # convert to numpy arrays and one-hot encode labels
    X = np.array(X)
    y = np.array(y)
    y_onehot = to_categorical(y, num_classes=num_classes)

    return X, y_onehot

In [ ]:
path = r"D:\Education\Semester 6\CSE 321 Project Based Learning on CSE\Composers Classification\Experiment 4\Mel-Spectrogram_aug"
X_test, y_test = preparing_data(path)

In [ ]:
path = "D:\Education\Semester 6\CSE 321 Project Based Learning on CSE\Composers Classification\Mel-Spectrogram\Train"
X_train, y_train = preparing_data(path)

In [ ]:
path = "D:\Education\Semester 6\CSE 321 Project Based Learning on CSE\Composers Classification\Experiment 4\Mel-Spectrogram_aug"
X_train_aug1, y_train_aug1 = preparing_data(path)

In [ ]:
path = r"D:\Education\Semester 6\CSE 321 Project Based Learning on CSE\Composers Classification\Experiment 4\X_train_aug1"
np.save(path, X_train_aug1)

path = r"D:\Education\Semester 6\CSE 321 Project Based Learning on CSE\Composers Classification\Experiment 4\y_train_aug1"
np.save(path, y_train_aug1)

In [2]:
X_train_aug2 = np.load(r"D:\Education\Semester 6\CSE 321 Project Based Learning on CSE\Composers Classification\Experiment 5\Preprocessed_masked\X_train.npy")
X_train_orig = np.load(r"D:\Education\Semester 6\CSE 321 Project Based Learning on CSE\Composers Classification\Experiment 5\Preprocessed_original\X_train.npy")
y_train_aug2 = np.load(r"D:\Education\Semester 6\CSE 321 Project Based Learning on CSE\Composers Classification\Experiment 5\Preprocessed_masked\y_train.npy")
y_train_orig = np.load(r"D:\Education\Semester 6\CSE 321 Project Based Learning on CSE\Composers Classification\Experiment 5\Preprocessed_original\y_train.npy")
X_train_aug1 = np.load(r"D:\Education\Semester 6\CSE 321 Project Based Learning on CSE\Composers Classification\Experiment 4\X_train_aug1.npy")
y_train_aug1 = np.load(r"D:\Education\Semester 6\CSE 321 Project Based Learning on CSE\Composers Classification\Experiment 4\y_train_aug1.npy")

In [3]:
print(X_train_orig.shape)
print(X_train_aug1.shape)
print(X_train_aug2.shape)
print(y_train_orig.shape)
print(y_train_aug1.shape)
print(y_train_aug2.shape)

print(X_test.shape)
print(y_test.shape)

print(X_val.shape)
print(y_val.shape)

(10886, 128, 216)
(41825, 128, 216)
(10886, 128, 216)
(10886, 8)
(41825, 8)
(10886, 8)
(2295, 128, 216)
(2295, 8)
(2199, 128, 216)
(2199, 8)


In [4]:
# Combine the two datasets and their labels
X_train1 = np.concatenate((X_train_aug1, X_train_aug2))
y_train1 = np.concatenate((y_train_aug1, y_train_aug2))

# Create an array of indices and shuffle them randomly
indices = np.arange(len(X_train1))
np.random.shuffle(indices)

# Shuffle the X_train and y_train arrays based on the shuffled indices
X_train1 = X_train1[indices]
y_train1 = y_train1[indices]

print(X_train1.shape)
print(y_train1.shape)

(52711, 128, 216)
(52711, 8)


In [5]:
# Combine the two datasets and their labels
X_train = np.concatenate((X_train1, X_train_orig))
y_train = np.concatenate((y_train1, y_train_orig))

# Create an array of indices and shuffle them randomly
indices = np.arange(len(X_train))
np.random.shuffle(indices)

# Shuffle the X_train and y_train arrays based on the shuffled indices
X_train = X_train[indices]
y_train = y_train[indices]

print(X_train.shape)
print(y_train.shape)

(63597, 128, 216)
(63597, 8)


In [2]:
X_train = np.load("D:\Education\Semester 6\CSE 321 Project Based Learning on CSE\Composers Classification\Experiment 5\X_train.npy")
y_train = np.load("D:\Education\Semester 6\CSE 321 Project Based Learning on CSE\Composers Classification\Experiment 5\y_train.npy")

X_test = np.load(r"D:\Education\Semester 6\CSE 321 Project Based Learning on CSE\Composers Classification\Experiment 5\Preprocessed_original\X_test.npy")
X_val = np.load(r"D:\Education\Semester 6\CSE 321 Project Based Learning on CSE\Composers Classification\Experiment 5\Preprocessed_original\X_val.npy")
y_test = np.load(r"D:\Education\Semester 6\CSE 321 Project Based Learning on CSE\Composers Classification\Experiment 5\Preprocessed_original\y_test.npy")
y_val = np.load(r"D:\Education\Semester 6\CSE 321 Project Based Learning on CSE\Composers Classification\Experiment 5\Preprocessed_original\y_val.npy")


# Model

In [3]:
model1 = models.Sequential()
model1.add(layers.Conv2D(filters=128, padding = "same", kernel_size=(5, 5), strides=(1, 1), activation=tf.nn.relu,input_shape=(128, 216, 1)))
model1.add(layers.MaxPool2D((2, 2),strides=2))
model1.add(layers.Dropout(0.3))

model1.add(layers.Conv2D(filters=128, padding = "same", kernel_size=(5, 5), strides=(1, 1), activation=tf.nn.relu))
model1.add(layers.MaxPool2D(pool_size=(2, 2),strides=2))
model1.add(layers.Dropout(0.3))

model1.add(layers.Conv2D(filters=128, padding = "same", kernel_size=(5, 5), strides=(1, 1), activation=tf.nn.relu))
model1.add(layers.MaxPool2D(pool_size=(2, 2),strides=2))
model1.add(layers.Dropout(0.3))

model1.add(layers.Conv2D(filters=128, padding = "same", kernel_size=(5, 5), strides=(1, 1), activation=tf.nn.relu))
model1.add(layers.MaxPool2D(pool_size=(2, 2),strides=2))
model1.add(layers.Dropout(0.3))

model1.add(layers.Conv2D(filters=128, padding = "same", kernel_size=(5, 5), strides=(1, 1), activation=tf.nn.relu))
model1.add(layers.MaxPool2D(pool_size=(2, 2),strides=2))
model1.add(layers.Dropout(0.3))

model1.add(layers.Conv2D(filters=128, padding = "same", kernel_size=(5, 5), strides=(1, 1), activation=tf.nn.relu))
model1.add(layers.MaxPool2D(pool_size=(2, 2),strides=2))
model1.add(layers.Dropout(0.3))

model1.add(layers.Flatten())
model1.add(layers.Dense(units=256, activation=tf.nn.relu,kernel_regularizer=tf.keras.regularizers.L2(0.003)))
model1.add(layers.Dense(units=128, activation=tf.nn.relu,kernel_regularizer=tf.keras.regularizers.L2(0.003)))
model1.add(layers.Dense(units=8, activation=tf.nn.softmax))

# define the checkpoint callback
checkpoint_path = "best_model1.h5"
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1,
                             save_best_only=True, mode='max')

# compile the model
model1.compile(loss="categorical_crossentropy",
             optimizer='adam',
             metrics=['accuracy'])

In [4]:
model1.fit(X_train, y_train, epochs=30 , batch_size=32, validation_data=(X_val, y_val), callbacks=[checkpoint])

Epoch 1/30
  22/1988 [..............................] - ETA: 1:24:04 - loss: 4.1121 - accuracy: 0.1108

KeyboardInterrupt: 